In [166]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm


import pandas as pd
import numpy as np
import requests
import time
import re

In [167]:
df = pd.read_csv('kakao_영등포구맛집(카카오api).csv')

In [168]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df.head()

,ID,stores,X,Y,road_address,place_url
0,20908598,공차 영등포구청점,126.897061,37.526699,서울 영등포구 당산로 136,http://place.map.kakao.com/20908598
1,1714644708,이도숙성회,126.896237,37.526695,서울 영등포구 당산로31길 4,http://place.map.kakao.com/1714644708
2,27527233,까치네분식,126.896689,37.527085,서울 영등포구 당산로 139,http://place.map.kakao.com/27527233
3,22246273,푸른농장 해바라기생고기정육식당 영등포점,126.896155,37.527045,서울 영등포구 당산로33길 5,http://place.map.kakao.com/22246273
4,19609040,사조참치 영등포구청점,126.896554,37.526706,서울 영등포구 당산로 133,http://place.map.kakao.com/19609040


In [169]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Chrome(options=options)

In [170]:
# 수집할 정보

menu_price = []
category = []
business_hours = [] 
rating = []
review_count = []
phone_number = [] 
last_update = []
delay = 0.1

In [171]:
for x in tqdm(df['place_url']):
    driver.get(x)
    time.sleep(1)
    try:
        business_hours.append(driver.find_element(By.CLASS_NAME, 'txt_operation').text) # 영업시간
        time.sleep(delay)
    except NoSuchElementException:
        business_hours.append('')
        time.sleep(delay)
    
    try:
        rating.append(driver.find_element(By.CLASS_NAME, 'color_b').text) # 별점
        time.sleep(delay)
    except NoSuchElementException:
        rating.append('')
        time.sleep(delay)
    
    try:
        category.append(driver.find_element(By.CLASS_NAME, 'txt_location').text) # 카테고리
        time.sleep(delay)
    except NoSuchElementException:
        category.append('')
        time.sleep(delay)
    
    try:
        review_count.append(re.sub('[()]', '', driver.find_element(By.CLASS_NAME, 'color_g').text)) # 리뷰수
        time.sleep(delay)
    except NoSuchElementException:
        review_count.append('')
        time.sleep(delay)
    
    try:
        phone_number.append(driver.find_element(By.CLASS_NAME, 'txt_contact').text) # 대표번호
        time.sleep(delay)
    except NoSuchElementException:
        phone_number.append('')
        time.sleep(delay)
    
    try:
        last_update.append(driver.find_element(By.CLASS_NAME, 'date_revise').text) # 최종 업데이트 날짜
        time.sleep(delay)
    except NoSuchElementException:
        last_update.append('')
        time.sleep(delay)
    
    # 메뉴 펼치기
    try:
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[3]/a/span[1]').click()
        time.sleep(delay)
    except NoSuchElementException:
        pass
    
    menu_temp = []
    price_temp = []
    # photo_menu 확인
    photo_menues = driver.find_elements(By.CLASS_NAME, 'photo_type')
    if len(photo_menues) == 0:
        pass
    else :
        for y in photo_menues:
            y = y.text.split('\n')
            try:
                menu_temp.append(y[0])
                price_temp.append(y[1])
            except:
                pass
    time.sleep(delay)
            
    normal_menues = driver.find_elements(By.CLASS_NAME, 'nophoto_type')
    if len(normal_menues) == 0:
        pass
    else :
        for z in normal_menues:
            z = z.text.split('\n')
            try:
                menu_temp.append(y[0])
                price_temp.append(y[1])
            except:
                pass
    menu_price.append({ key : value for key, value in zip(menu_temp, price_temp)})
    time.sleep(1)

100%|██████████| 1412/1412 [1:13:27<00:00,  3.12s/it]


In [173]:
driver.close()

In [174]:
df['menu_price'] = menu_price
df['category'] = category
df['business_hours'] = business_hours 
df['rating'] = rating
df['review_count'] = review_count
df['phone_number'] = phone_number
df['last_update'] = last_update
df.to_csv('카카오API_크롤링(selenium).csv', encoding='utf-8-sig', index=False)

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412 entries, 0 to 1411
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1412 non-null   int64  
 1   stores          1412 non-null   object 
 2   X               1412 non-null   float64
 3   Y               1412 non-null   float64
 4   road_address    1412 non-null   object 
 5   place_url       1412 non-null   object 
 6   menu_price      1412 non-null   object 
 7   category        1412 non-null   object 
 8   business_hours  1412 non-null   object 
 9   rating          1412 non-null   object 
 10  review_count    1412 non-null   object 
 11  phone_number    1412 non-null   object 
 12  last_update     1412 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 143.5+ KB


In [177]:
df.tail()

,ID,stores,X,Y,road_address,place_url,menu_price,category,business_hours,rating,review_count,phone_number,last_update
1407,15681007,엄니식당,126.931091,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/15681007,{},한식,,2.8,5,02-785-1007,2022.04.05.
1408,560071396,머글거리,126.935073,37.528453,서울 영등포구 여의동로 338,http://place.map.kakao.com/560071396,"{'우육면': '8,000'}",분식,,1.5,2,070-4272-8160,2021.04.17.
1409,8279880,다락방청국장,126.931091,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/8279880,{},한식,,3.0,Darakbangcheonggukjang,02-780-2009,2022.07.08.
1410,758847688,쿡스테이크,126.932853,37.527045,서울 영등포구 여의동로 지하 343,http://place.map.kakao.com/758847688,{},"스테이크,립",,0,0,010-3047-6687,2019.12.16.
1411,1844515946,미스터리키친,126.932974,37.527133,서울 영등포구 여의동로 지하 343,http://place.map.kakao.com/1844515946,{},패스트푸드,,0,0,010-7773-0960,2019.12.16.


In [179]:
df.loc[df['menu_price']=={}, 'menu_price'] = np.NaN

In [181]:
df.loc[df['business_hours']=='', 'business_hours'] = np.NaN

In [182]:
df['review_count'].value_counts()

0                         383
1                         110
2                          84
3                          72
4                          66
                         ... 
Paik's Coffee               1
CROME                       1
CAFE EL                     1
Cocohodo                    1
Darakbangcheonggukjang      1
Name: review_count, Length: 321, dtype: int64

In [183]:
df['place_url'][1409]

'http://place.map.kakao.com/8279880'

In [184]:
df.tail()

,ID,stores,X,Y,road_address,place_url,menu_price,category,business_hours,rating,review_count,phone_number,last_update
1407,15681007,엄니식당,126.931091,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/15681007,NaN,한식,NaN,2.8,5,02-785-1007,2022.04.05.
1408,560071396,머글거리,126.935073,37.528453,서울 영등포구 여의동로 338,http://place.map.kakao.com/560071396,"{'우육면': '8,000'}",분식,NaN,1.5,2,070-4272-8160,2021.04.17.
1409,8279880,다락방청국장,126.931091,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/8279880,NaN,한식,NaN,3.0,Darakbangcheonggukjang,02-780-2009,2022.07.08.
1410,758847688,쿡스테이크,126.932853,37.527045,서울 영등포구 여의동로 지하 343,http://place.map.kakao.com/758847688,NaN,"스테이크,립",NaN,0,0,010-3047-6687,2019.12.16.
1411,1844515946,미스터리키친,126.932974,37.527133,서울 영등포구 여의동로 지하 343,http://place.map.kakao.com/1844515946,NaN,패스트푸드,NaN,0,0,010-7773-0960,2019.12.16.
